In [127]:
import re #pacchetto per espressioni regolari
import os #pacchetto per muoversi nelle cartelle
import sys
sys.path.append('..')  # Add parent directory to the Python path
from utils.helpers import *


In [139]:
import pandas as pd
children = pd.read_csv("children/children.csv")
children_test = pd.read_csv("children/children_test.csv")
twitter = pd.read_csv("twitter/twitter.csv")
twitter_test = pd.read_csv("twitter/twitter_test.csv")
youtube = pd.read_csv("youtube/youtube.csv")
youtube_test = pd.read_csv("youtube/youtube_test.csv")
diary = pd.read_csv("diary/diary.csv")
diary_test = pd.read_csv("diary/diary_test.csv")

In [140]:
training_set = pd.concat([children, twitter, youtube, diary])

In [141]:
test_set = pd.concat([children_test, twitter_test, youtube_test, diary_test])

In [ ]:
# Sostituire i NaN nelle colonne "136" e "137" con 0.0
#training_set[["136", "137"]] = training_set[["136", "137"]].fillna(0.0)
#test_set[["136", "137"]] = test_set[["136", "137"]].fillna(0.0)

# Sostituire i NaN nelle colonne con la media
training_set['136'] = training_set['136'].fillna(training_set['136'].mean())
training_set['137'] = training_set['137'].fillna(training_set['137'].mean())
training_set['138'] = training_set['138'].fillna(training_set['138'].mean())
training_set['139'] = training_set['139'].fillna(training_set['139'].mean())
training_set['140'] = training_set['140'].fillna(training_set['140'].mean())



In [143]:
test_set['136'] = test_set['136'].fillna(test_set['136'].mean())
test_set['137'] = test_set['137'].fillna(test_set['137'].mean())
test_set['138'] = test_set['138'].fillna(test_set['138'].mean())
test_set['139'] = test_set['139'].fillna(test_set['139'].mean())
test_set['140'] = test_set['140'].fillna(test_set['140'].mean())


In [144]:
X_train = training_set.drop(columns=['Label', 'genre']).values
train_labels = training_set["Label"].tolist()


X_test = test_set.drop(columns=['Label', 'genre']).values
test_labels = test_set["Label"].tolist()


In [145]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search through
param_grid = {
    'C': [10, 1, 0.1, 0.01, 0.001, 0.0001],  # Regularization parameter
    'dual': [True, False]
}

# Initialize SVM with linear kernel
svc = LinearSVC()

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5)

# Fit the grid search to your data
grid_search.fit(X_train, train_labels)

# Get mean test scores across folds
mean_test_scores = grid_search.cv_results_['mean_test_score']

# Print the best parameters and best score
print("Best parameters found:", grid_search.best_params_)
print("Best score found:", grid_search.best_score_)

c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters found: {'C': 10, 'dual': True}
Best score found: 0.5798054654932839


c:\Users\corra\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [146]:
# Get the best estimator (model) found by grid search
best_model = grid_search.best_estimator_

# Now, you can use this best_model to make predictions on new data
# For example, if you have new data X_new, you can predict its labels as follows:
predictions = best_model.predict(X_test)

In [147]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix
test_predictions = predictions
print(classification_report(test_labels, test_predictions, zero_division=0)) # output_dict=True

              precision    recall  f1-score   support

           F       0.53      0.66      0.59      5337
           M       0.55      0.41      0.47      5337

    accuracy                           0.54     10674
   macro avg       0.54      0.54      0.53     10674
weighted avg       0.54      0.54      0.53     10674



In [148]:
from sklearn.metrics import accuracy_score, classification_report

# Aggiungi le predizioni come una nuova colonna nel test_set
test_set['predicted'] = test_predictions

# Lista delle categorie da analizzare
genres = ['children_test', 'twitter_test', 'youtube_test', 'diary_test']

# Ciclo per calcolare le metriche per ogni categoria
for genre in genres:
    # Filtra per categoria
    genre_data = test_set[test_set['genre'] == genre]
    genre_labels = genre_data['Label']
    genre_predictions = genre_data['predicted']
    
    # Calcola le metriche per la categoria
    accuracy = accuracy_score(genre_labels, genre_predictions)
    report = classification_report(genre_labels, genre_predictions)
    
    # Mostra i risultati
    print(f"{genre.capitalize()} Accuracy: {accuracy}")
    print(f"{genre.capitalize()} Classification Report:\n{report}")
    print()


Children_test Accuracy: 0.53
Children_test Classification Report:
              precision    recall  f1-score   support

           F       0.53      0.55      0.54       100
           M       0.53      0.51      0.52       100

    accuracy                           0.53       200
   macro avg       0.53      0.53      0.53       200
weighted avg       0.53      0.53      0.53       200


Twitter_test Accuracy: 0.5476666666666666
Twitter_test Classification Report:
              precision    recall  f1-score   support

           F       0.54      0.66      0.59      3000
           M       0.56      0.43      0.49      3000

    accuracy                           0.55      6000
   macro avg       0.55      0.55      0.54      6000
weighted avg       0.55      0.55      0.54      6000


Youtube_test Accuracy: 0.5256818181818181
Youtube_test Classification Report:
              precision    recall  f1-score   support

           F       0.52      0.68      0.59      2200
           M 